In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
import gym
from gym import spaces
from sklearn.preprocessing import MinMaxScaler
from collections import deque
import random

battery_file_path = '/content/battery.csv'
engine_file_path = '/content/power.csv'
suspension_file_path = '/content/suspension.csv'

battery_data = pd.read_csv(battery_file_path)
engine_data = pd.read_csv(engine_file_path)
suspension_data = pd.read_csv(suspension_file_path)

print("Battery Data Sample:\n", battery_data.head())
print("Engine Data Sample:\n", engine_data.head())
print("Suspension Data Sample:\n", suspension_data.head())

# Step 3: Data Preprocessing
scaler = MinMaxScaler()

merged_data = pd.merge(battery_data, engine_data, on='Timestamp')
merged_data = pd.merge(merged_data, suspension_data, on='Timestamp')

features = merged_data.drop(columns=['Timestamp'])
scaled_features = scaler.fit_transform(features)

scaled_df = pd.DataFrame(scaled_features, columns=features.columns)

# Step 4: Define the Custom Environment
class CarMaintenanceEnv(gym.Env):
    def __init__(self, data):
        super(CarMaintenanceEnv, self).__init__()
        self.data = data
        self.current_step = 0

        # Define action and observation space
        self.action_space = spaces.Discrete(2)  # 0 = No alert, 1 = Trigger alert
        self.observation_space = spaces.Box(low=0, high=1, shape=(scaled_df.shape[1]-1,), dtype=np.float32)

    def reset(self):
        self.current_step = 0
        return self.data.iloc[self.current_step, :-1].values

    def step(self, action):
        state = self.data.iloc[self.current_step, :-1].values
        actual_alert = self.data.iloc[self.current_step, -1]
        self.current_step += 1

        # Calculate reward
        reward = 10 if action == actual_alert else -10
        done = self.current_step >= len(self.data) - 1

        return state, reward, done, {}

    def render(self, mode="human"):
        pass

env = CarMaintenanceEnv(data=scaled_df)

# Step 5: Define the DQN Model
class DQN:
    def __init__(self, env, learning_rate=0.001):
        self.env = env
        self.learning_rate = learning_rate  # Initialize learning_rate here
        self.model = self.create_model()
        self.target_model = self.create_model()
        self.update_target_model()

        # Hyperparameters
        self.memory = deque(maxlen=2000)
        self.gamma = 0.95
        self.epsilon = 1.0
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.batch_size = 32

    def create_model(self):
        # Define the neural network architecture
        model = tf.keras.Sequential([
            layers.Dense(24, input_shape=(self.env.observation_space.shape[0],), activation='relu'),
            layers.Dense(24, activation='relu'),
            layers.Dense(self.env.action_space.n, activation='linear')
        ])
        # Compile the model with the specified learning rate
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=self.learning_rate), loss='mse')
        return model

    def update_target_model(self):
        self.target_model.set_weights(self.model.get_weights())

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.env.action_space.n)
        q_values = self.model.predict(state)
        return np.argmax(q_values[0])

    def replay(self):
        if len(self.memory) < self.batch_size:
            return
        minibatch = random.sample(self.memory, self.batch_size)
        for state, action, reward, next_state, done in minibatch:
            target = reward
            if not done:
                target += self.gamma * np.amax(self.target_model.predict(next_state)[0])
            target_f = self.model.predict(state)
            target_f[0][action] = target
            self.model.fit(state, target_f, epochs=1, verbose=0)
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay

    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))

    def train(self, episodes=100):
        for e in range(episodes):
            state = self.env.reset()
            state = np.reshape(state, [1, self.env.observation_space.shape[0]])
            total_reward = 0
            for time in range(500):
                action = self.act(state)
                next_state, reward, done, _ = self.env.step(action)
                next_state = np.reshape(next_state, [1, self.env.observation_space.shape[0]])
                self.remember(state, action, reward, next_state, done)
                state = next_state
                total_reward += reward
                if done:
                    print(f"Episode: {e + 1}/{episodes}, Total Reward: {total_reward}, Epsilon: {self.epsilon:.2f}")
                    break
                self.replay()
            self.update_target_model()


# Step 6: Train the Model
dqn_agent = DQN(env)
dqn_agent.train(episodes=100)

# Step 7: Save the Trained Model
dqn_agent.model.save("car_maintenance_model.h5")
print("Training complete and model saved as 'car_maintenance_model.h5'")


Battery Data Sample:
           Timestamp  Battery Voltage (V)  Battery Current (A)  \
0  11-11-2023 12:00                 12.6                   10   
1  11-11-2023 12:01                 12.5                   15   
2  11-11-2023 12:02                 12.4                    8   

   Battery Temperature (°C)  Ambient Temperature (°C)  Vehicle Speed (km/h)  \
0                        30                        25                    60   
1                        35                        25                     0   
2                        32                        25                    80   

   Vehicle Acceleration (m/s²)  Vehicle Braking  
0                            2                0  
1                            0                1  
2                            3                0  
Engine Data Sample:
           Timestamp  Vehicle Speed (km/h)  Engine RPM (RPM)  \
0  11-11-2023 12:00                    60              2000   
1  11-11-2023 12:01                     0            

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Episode: 14/100, Total Reward: 0, Epsilon: 1.00
Episode: 15/100, Total Reward: -20, Epsilon: 1.00
Episode: 16/100, Total Reward: 0, Epsilon: 1.00
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 89ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━

Episode: 100/100, Total Reward: 0, Epsilon: 0.66
Training complete and model saved as 'car_maintenance_model.h5'
